# Testing Findata.py library as a potential dependency

I've been looking for the right software to bridge the gap between getting data and what I'm working on. In the past I normally write the code to gather and prepare the data but I was always annoyed by my ad-hoc work and concerned it didn't follow standard practices. I came across the findata.py library and now want to see if it's the right dependency for PyPort.

Quick blerb:
findata.py is part of the Cuemacro Python libraries and has a number of dependencies of their own. For now I'm just interested in findata.py itself. One thing at a time...

For now I'll focus on using Yahoo as the data provider (`data_source`)

***
Example: Getting Equity Data
***
In the 'findata' repo, the author provided an example of getting equity data from different providers. I recreated some of that for this notebook. Here I pull the past decade of Apple and Citigroup stock closing stock prices for the past 10 years.

In [74]:
# what we will need
import findatapy
import pandas as pd
from findatapy.util import SwimPool; SwimPool()

market = Market(market_data_generator=MarketDataGenerator())

# extra stuff to make the notebook easier to use
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [161]:
# download equities data from Yahoo
md_request = MarketDataRequest(     # md_request is short for 'market data request'
    start_date="decade",            # start date
    data_source='yahoo',            # use Bloomberg as data source
    tickers=['Apple', 'Citigroup'], # ticker (findatapy)
    fields=['close'],               # which fields to download
    vendor_tickers=['aapl', 'c'],   # ticker (Yahoo)
    vendor_fields=['Close'])        # which Bloomberg fields to download)

In [160]:
results = market.fetch_market(md_request)

2019-11-26 13:00:14,871 - findatapy.market.datavendorweb - INFO - Request Yahoo data
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
2019-11-26 13:00:15,663 - findatapy.market.datavendorweb - INFO - Completed request from Yahoo.
2019-11-26 13:00:17,674 - findatapy.market.ioengine - WARNING - Couldn't push MarketDataRequest_412__abstract_curve_key-None__category-None__category_key-backtest_default-cat_yahoo_daily_NYC__cut-NYC__data_source-yahoo__environment-backtest__expiry_date-NaT__fields-close__finish_date-2019-11-24 21:01:32_587982__freq-daily__freq_mult-1__gran_freq-None__resample-None__resample_how-last__start_date-2009-11-26 23:01:22_374933__tickers-Apple_Citigroup__trade_side-trade__vendor_fields-Close__vendor_tickers-aapl_c to Redis: Error 10061 connecting to 127.0.0.1:6379. No connection could be made because the target machine actively refused it.


### A note:
First, the terminal says that the download was successful. You can see that by the bar with 100% in it. But what is up with that next line? Something about findatapy.market.ioengine... What the heck? 

### Explaination:

What you are seeing here is nothing to be worried about. It's looking to send a hash key (the big jumbled mess of words) to a program called [Redis](https://redis.io/topics/introduction). But since I'm not running Redis on my windows machine I get that error. Without Redis my machine refuses the push. That being said, you don't need Redis at all, especially at the level we are working with. That's for a future topic.

So, was `findata` able to successfully download the past decade of apple and citigroup data? What are we working with here?

In [190]:
type(results)
results.shape
results.isnull().sum().sum()
results.head(3)
results.tail(4)

pandas.core.frame.DataFrame

(2515, 2)

0

,Apple.close,Citigroup.close
Date,,
2009-11-27,28.655714,40.599998
2009-11-30,28.558571,41.099998
2009-12-01,28.138571,41.000000


,Apple.close,Citigroup.close
Date,,
2019-11-19,266.290009,74.820000
2019-11-20,263.190002,73.910004
2019-11-21,262.010010,73.900002
2019-11-22,261.779999,74.870003



### Yep, it worked.
***


# Discussion on the writable attributes for the MarketDataRequest Object 

(at least the ones we care about)

It took some playing around in this notebook (see towards the end) and looking at the source code to understand all the ins and outs of the `MarketDataRequest` Object. I think I have a good grasp on what is going on...

#### data_source
> to do

#### start_date
> to do

#### finish_date
> to do

#### tickers
> to do

#### category
> to do

#### freq_mult
> to do

#### freq
> to do

#### vendor_tickers
> to do

#### vendor_fields
> to do

***
# More testing
Let's try my silly portfolio's ticker list and pull data from 2015 onwards. But before I try it using findata I'll use my ad-hoc code first. I'm currious to see how they stack up together. For the next couple of cells I'm working with daily adjusted closing data for 37 equities between 1st January 2015 and today (when I run this).

In [112]:
# going to need this.
import pandas_datareader as pdr

# set up list of tickers in my silly portfolio
ticker_list = ['LOW','BKNG','DD','AMT','GE','RSG','SNPS','VEEV','CBS','DXC','BG','LAMR','KIM','HELE','SIGI','S','FHB','CRS','AIMC','TEX','FORM','CSGS','AROC','ATRA','ANDE','TELL','GFF','LMNX','ACLS','GOOD','AKBA','RILY','TLRA','LAND','FBMS','NKSH','TWIN']

In [118]:
# My code that I've been using
df_from_pdr = pdr.get_data_yahoo(ticker_list, '2015', '2020', interval = 'd')  # df starts with multindex 
df_from_pdr = df_from_pdr.loc[:, pd.IndexSlice['Adj Close', :]] # overwrite so each stock only has adjusted close
df_from_pdr.columns = df_from_pdr.columns.levels[1] # get rid of multindex


That seemed to take forever (at least when I ran it). Just for clarity, here's what we got back.

In [191]:
pd.set_option('display.max_columns', 50)
df_from_pdr.shape
df_from_pdr.isna().sum().sum()
df_from_pdr.head(3)
df_from_pdr.tail(3)

(1234, 37)

0

Symbols,ACLS,AIMC,AKBA,AMT,ANDE,AROC,ATRA,BG,BKNG,CBS,CRS,CSGS,DD,DXC,FBMS,FHB,FORM,GE,GFF,GOOD,HELE,KIM,LAMR,LAND,LMNX,LOW,NKSH,RILY,RSG,S,SIGI,SNPS,TELL,TEX,TLRA,TWIN,VEEV
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-02,10.2,26.016121,11.50,90.685654,48.438496,26.169426,26.700001,79.203506,1142.060059,51.564766,43.914070,22.245516,58.773994,21.410477,13.919315,622.100403,8.31,20.850058,11.757659,11.645465,64.790001,19.39748,42.702259,8.350659,18.248041,61.919804,26.120977,8.900008,36.206093,4.24,24.898741,43.099998,7.36,26.873728,2.89,18.602312,26.230000
2015-01-05,10.0,25.067421,11.09,89.375458,48.147846,25.318903,27.780001,77.596153,1097.579956,51.065964,41.540085,21.763500,56.956245,20.977194,13.765296,615.834534,8.45,20.467335,11.633522,11.679143,63.330002,19.57272,42.130466,8.101983,18.151079,60.438099,25.359257,8.371297,36.000828,4.21,24.526976,42.540001,6.64,25.135670,2.83,17.934141,26.230000
2015-01-06,9.6,24.492735,11.06,89.138893,47.185062,24.421118,25.889999,76.866318,1079.959961,50.058945,40.724319,21.708931,56.040916,21.031355,13.505394,617.624695,8.01,20.026371,11.482782,11.638725,63.360001,19.68700,42.456066,8.182197,17.908678,60.575279,25.667370,8.371297,36.125778,4.19,24.071566,42.160000,6.80,24.323320,2.65,17.362804,27.120001


Symbols,ACLS,AIMC,AKBA,AMT,ANDE,AROC,ATRA,BG,BKNG,CBS,CRS,CSGS,DD,DXC,FBMS,FHB,FORM,GE,GFF,GOOD,HELE,KIM,LAMR,LAND,LMNX,LOW,NKSH,RILY,RSG,S,SIGI,SNPS,TELL,TEX,TLRA,TWIN,VEEV
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-11-21,21.059999,33.049999,3.460,213.539993,23.070000,8.670,13.74,54.639999,1873.310059,38.98,51.799999,57.41,65.690002,35.610001,33.130001,28.150000,22.219999,11.530,21.780001,22.2500,153.029999,21.340000,82.690002,12.11,20.219999,117.019997,46.900002,29.020000,86.750000,5.630,64.739998,138.770004,7.68,27.430000,8.120,10.010,150.660004
2019-11-22,20.719999,33.259998,4.210,210.110001,23.570000,8.600,13.84,54.080002,1877.530029,39.52,51.770000,56.34,65.830002,35.840000,33.230000,28.290001,22.520000,11.550,21.510000,21.9800,155.270004,21.299999,82.889999,12.10,20.660000,118.199997,45.930000,29.059999,87.250000,5.740,64.570000,135.380005,7.51,28.059999,8.220,10.040,153.270004
2019-11-25,21.420000,33.759998,4.469,209.880005,23.860001,8.785,14.03,54.189999,1890.430054,39.52,53.257999,56.98,65.339996,36.189999,33.439999,28.379999,23.014999,11.705,22.219999,22.0935,158.899002,21.475000,82.985001,12.17,21.270000,116.849998,43.480099,29.450001,87.779999,5.875,66.059998,137.740005,7.82,28.809999,8.445,10.295,156.600006


Now onto using findata for the same task. Let's also get the adjusted close prices. And this time I'm not going to use `ticker` argument and instead rely on the `vendor_tickers` argument. It should still work.

In [114]:
# set up a MarketDataRequest object:
portfolio_data_object = MarketDataRequest(
    data_source= 'yahoo',
    start_date = 'Jan 01 2015',
    fields= ['adj close'],
    vendor_tickers = ticker_list,
    vendor_fields=['adj close'])
df = market.fetch_market(portfolio_data_object)

2019-11-25 11:33:26,794 - findatapy.market.datavendorweb - INFO - Request Yahoo data
[*********************100%***********************]  37 of 37 completed
[*********************100%***********************]  37 of 37 completed
2019-11-25 11:33:31,049 - findatapy.market.datavendorweb - INFO - Completed request from Yahoo.
2019-11-25 11:33:33,073 - findatapy.market.ioengine - WARNING - Couldn't push MarketDataRequest_561__abstract_curve_key-None__category-None__category_key-backtest_default-cat_yahoo_daily_NYC__cut-NYC__data_source-yahoo__environment-backtest__expiry_date-NaT__fields-adj close__finish_date-2019-11-24 21:01:32_587982__freq-daily__freq_mult-1__gran_freq-None__resample-None__resample_how-last__start_date-2015-01-01 00:00:00__tickers-None__trade_side-trade__vendor_fields-adj close__vendor_tickers-LOW_BKNG_DD_AMT_GE_RSG_SNPS_VEEV_CBS_DXC_BG_LAMR_KIM_HELE_SIGI_S_FHB_CRS_AIMC_TEX_FORM_CSGS_AROC_ATRA_ANDE_TELL_GFF_LMNX_ACLS_GOOD_AKBA_RILY_TLRA_LAND_FBMS_NKSH_TWIN to Redis: Error

Wow that was much faster than my code. Did I get the same results?

In [103]:
df.shape
df.isnull().sum().sum() # this will tell me how much data is missing
df.head(3)
df.tail(3)

(1233, 37)

0

,LOW.adj close,BKNG.adj close,DD.adj close,AMT.adj close,GE.adj close,RSG.adj close,SNPS.adj close,VEEV.adj close,CBS.adj close,DXC.adj close,BG.adj close,LAMR.adj close,KIM.adj close,HELE.adj close,SIGI.adj close,S.adj close,FHB.adj close,CRS.adj close,AIMC.adj close,TEX.adj close,FORM.adj close,CSGS.adj close,AROC.adj close,ATRA.adj close,ANDE.adj close,TELL.adj close,GFF.adj close,LMNX.adj close,ACLS.adj close,GOOD.adj close,AKBA.adj close,RILY.adj close,TLRA.adj close,LAND.adj close,FBMS.adj close,NKSH.adj close,TWIN.adj close
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-02,61.919804,1142.060059,58.773994,90.685654,20.850058,36.206093,43.099998,26.230000,51.564766,21.410477,79.203506,42.702259,19.39748,64.790001,24.898741,4.24,622.100403,43.914070,26.016121,26.873728,8.31,22.245516,26.169426,26.700001,48.438496,7.36,11.757659,18.248041,10.2,11.645465,11.50,8.900008,2.89,8.350659,13.919315,26.120977,18.602312
2015-01-05,60.438099,1097.579956,56.956245,89.375458,20.467335,36.000828,42.540001,26.230000,51.065964,20.977194,77.596153,42.130466,19.57272,63.330002,24.526976,4.21,615.834534,41.540085,25.067421,25.135670,8.45,21.763500,25.318903,27.780001,48.147846,6.64,11.633522,18.151079,10.0,11.679143,11.09,8.371297,2.83,8.101983,13.765296,25.359257,17.934141
2015-01-06,60.575279,1079.959961,56.040916,89.138893,20.026371,36.125778,42.160000,27.120001,50.058945,21.031355,76.866318,42.456066,19.68700,63.360001,24.071566,4.19,617.624695,40.724319,24.492735,24.323320,8.01,21.708931,24.421118,25.889999,47.185062,6.80,11.482782,17.908678,9.6,11.638725,11.06,8.371297,2.65,8.182197,13.505394,25.667370,17.362804


,LOW.adj close,BKNG.adj close,DD.adj close,AMT.adj close,GE.adj close,RSG.adj close,SNPS.adj close,VEEV.adj close,CBS.adj close,DXC.adj close,BG.adj close,LAMR.adj close,KIM.adj close,HELE.adj close,SIGI.adj close,S.adj close,FHB.adj close,CRS.adj close,AIMC.adj close,TEX.adj close,FORM.adj close,CSGS.adj close,AROC.adj close,ATRA.adj close,ANDE.adj close,TELL.adj close,GFF.adj close,LMNX.adj close,ACLS.adj close,GOOD.adj close,AKBA.adj close,RILY.adj close,TLRA.adj close,LAND.adj close,FBMS.adj close,NKSH.adj close,TWIN.adj close
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-11-20,117.830002,1851.819946,65.050003,217.899994,11.39,87.709999,140.839996,154.330002,39.009998,35.889999,55.299999,82.949997,21.440001,155.000000,65.660004,5.63,28.204496,52.459999,32.980000,27.440001,22.709999,57.91,8.54,14.22,22.66,7.95,22.049999,20.320000,22.170000,22.58,3.34,28.590000,8.10,12.14,33.049999,44.969635,10.07
2019-11-21,117.019997,1873.310059,65.690002,213.539993,11.53,86.750000,138.770004,150.660004,38.980000,35.610001,54.639999,82.690002,21.340000,153.029999,64.739998,5.63,28.150000,51.799999,33.049999,27.430000,22.219999,57.41,8.67,13.74,23.07,7.68,21.780001,20.219999,21.059999,22.25,3.46,29.020000,8.12,12.11,33.130001,46.900002,10.01
2019-11-22,118.199997,1877.530029,65.830002,210.110001,11.55,87.250000,135.380005,153.270004,39.520000,35.840000,54.080002,82.889999,21.299999,155.270004,64.570000,5.74,28.290001,51.770000,33.259998,28.059999,22.520000,56.34,8.60,13.84,23.57,7.51,21.510000,20.660000,20.719999,21.98,4.21,29.059999,8.22,12.10,33.230000,45.930000,10.04


Those are the same results with a couple caveats. findata didn't return today's adjusted closing price while my pandas data reader did. Which might sound bad but as I write this the market is still open. So there's something I didn't know. Pandas data reader `.get_data_yahoo` returned what appears to be non-historical data (albeit, just one row.)

Another caveat is the column index names returned by findata. I like that the price measurement field is in the column name. At a glance it tells me what I am looking at.

I've got the feeling that findata.py might just be the library I was looking for. Some things I like about it.
* It was built with pandas dataframe in mind
* It can handle a number of different datavenders (some I have access to, some I don't)
* It is super fast compared to my ad-hoc data gathering code.
* It handles errors more gracefully than anything I could come up with.
* It has plenty of additional features
* it handles multiple different date formates gracefully.
* Provides info when an error is encountered.

Some things I don't like but can overlook.

* it doesn't make it clear that for the `ticker` argument you must pass the name of the company
* but you can bypass that by passing a list of ticker symbols in `vender_tickers` which I like.
* I can't get it to handle simple dates such as the year `2012` or the like.
* I can't figure out how to supress the Redis cache key generation. I don't use Redis at the moment so I'd like to supress that.
* The results can be quirky if you're not careful with your inputs (see below)
***
# Let's get some more data and see when things break.

Below I have a new ticker list randomly created. I know that some these tickers will cause problems for pandas data reader because they came into existance in the past decade. I also know that one of these tickers, 'CHSP' is not an equity but actually a REIT. Let's see what happens first with my code.

In [125]:
ticker_list_2nd = ['MLVF','APYX','ICPT','RH','BMCH','CETV','GLUU','COLB','APPN','PCYO','OPI','ENS','ACIW','BTU','IRT','AVCO','GDP','AVX','HMSY','MGRC','SAIC','BH','PLOW','OOMA','GEOS','BRT','GLOG','RGR','TIPT','SIGA','LRN','ANF','DCOM','BDGE','PTGX','FELE','NPK','FORR','MSGN','SSD','CFMS','BXG','UNT','TRUP','HMTV','ACRE','FSBW','WIFI','CNO','CHSP']

# will this work?
print("Building Main DataFrame")
df_bad = pdr.get_data_yahoo(ticker_list_2nd, '2009', '2020', interval = 'd')  # df starts with multindex 
df_bad = df_bad.loc[:, pd.IndexSlice['Adj Close', :]] # overwrite so each stock only has adjusted close
df_bad.columns = df_bad.columns.levels[1] # get rid of multindex
print("Done!")

Building Main DataFrame
Done!
C:\Users\timot\Anaconda3\lib\site-packages\pandas_datareader\base.py:230: SymbolWarning: Failed to read symbol: 'CHSP', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)


I'm suprised that worked. Because I just ran this in another window and it failed. What the heck?

Wait, if I change the end date to 2019 it fails. But if I change it back to 2020 it suddenly works... what gives?

I'll have to investigate this some other time.

Anyway, what are we working with here?

In [129]:
df_bad.shape
df_bad.isna().sum().sum()
df_bad.head().round(3)
df_bad.tail().round(3)

(2744, 50)

31997

Symbols,ACIW,ACRE,ANF,APPN,APYX,AVCO,AVX,BDGE,BH,BMCH,BRT,BTU,BXG,CETV,CFMS,CHSP,CNO,COLB,DCOM,ENS,FELE,FORR,FSBW,GDP,GEOS,GLOG,GLUU,HMSY,HMTV,ICPT,IRT,LRN,MGRC,MLVF,MSGN,NPK,OOMA,OPI,PCYO,PLOW,PTGX,RGR,RH,SAIC,SIGA,SSD,TIPT,TRUP,UNT,WIFI
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2009-01-02,5.357,NaN,17.698,NaN,NaN,NaN,6.110,13.202,106.386,NaN,3.750,NaN,2.857,4.602,NaN,NaN,4.188,8.615,8.762,10.568,12.643,22.650,NaN,NaN,10.230,NaN,0.49,10.523,NaN,NaN,NaN,18.71,15.196,8.311,NaN,38.003,NaN,NaN,2.57,NaN,NaN,4.802,NaN,NaN,3.21,23.225,NaN,NaN,28.63,NaN
2009-01-05,5.417,NaN,17.858,NaN,NaN,NaN,5.973,12.681,115.524,NaN,3.426,NaN,2.665,4.363,NaN,NaN,3.957,8.170,8.608,10.587,12.963,21.128,NaN,NaN,10.875,NaN,0.50,10.390,NaN,NaN,NaN,18.74,15.646,8.130,NaN,37.833,NaN,NaN,2.50,NaN,NaN,4.833,NaN,NaN,3.28,22.519,NaN,NaN,29.87,NaN
2009-01-06,5.543,NaN,17.828,NaN,NaN,NaN,6.102,13.350,120.180,NaN,3.606,NaN,3.105,4.563,NaN,NaN,4.170,8.659,8.929,10.811,13.191,21.708,NaN,NaN,11.490,NaN,0.76,10.233,NaN,NaN,NaN,17.18,16.481,8.130,NaN,39.004,NaN,NaN,2.95,NaN,NaN,4.940,NaN,NaN,3.28,22.015,NaN,NaN,30.53,NaN
2009-01-07,5.570,NaN,17.247,NaN,NaN,NaN,5.996,12.701,115.179,NaN,3.354,NaN,3.151,4.191,NaN,NaN,3.877,8.396,8.702,10.446,12.722,21.024,NaN,NaN,10.485,NaN,0.70,10.150,NaN,NaN,NaN,16.02,15.990,8.130,NaN,38.679,NaN,NaN,2.95,NaN,NaN,5.093,NaN,NaN,3.54,21.015,NaN,NaN,29.46,NaN
2009-01-08,5.750,NaN,16.644,NaN,NaN,NaN,5.950,12.850,119.662,NaN,3.267,NaN,3.315,4.237,NaN,NaN,3.912,8.279,8.982,10.259,12.818,20.903,NaN,NaN,10.135,NaN,0.70,10.130,NaN,NaN,NaN,16.51,16.833,8.130,NaN,38.912,NaN,NaN,2.64,NaN,NaN,5.101,NaN,NaN,3.59,20.831,NaN,NaN,30.43,NaN


Symbols,ACIW,ACRE,ANF,APPN,APYX,AVCO,AVX,BDGE,BH,BMCH,BRT,BTU,BXG,CETV,CFMS,CHSP,CNO,COLB,DCOM,ENS,FELE,FORR,FSBW,GDP,GEOS,GLOG,GLUU,HMSY,HMTV,ICPT,IRT,LRN,MGRC,MLVF,MSGN,NPK,OOMA,OPI,PCYO,PLOW,PTGX,RGR,RH,SAIC,SIGA,SSD,TIPT,TRUP,UNT,WIFI
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-11-19,36.39,15.550,16.800,43.560,6.750,1.570,14.84,33.00,105.23,29.05,17.170,9.06,9.58,4.47,1.80,NaN,18.060,39.89,20.190,68.65,53.31,37.61,59.38,9.21,14.92,8.92,5.350,28.75,12.94,85.55,14.750,19.660,75.11,23.380,17.90,89.420,11.42,32.17,11.86,54.00,10.64,45.39,182.22,84.03,5.08,80.980,7.89,34.43,0.700,11.35
2019-11-20,36.40,15.550,15.930,44.060,7.080,1.540,14.61,32.80,107.06,29.14,17.800,8.96,9.57,4.46,1.77,NaN,17.890,39.45,20.030,68.69,53.09,37.51,56.57,9.16,14.79,9.17,5.400,29.00,12.95,86.59,14.740,19.740,73.31,22.840,17.08,88.150,11.11,32.39,12.07,52.86,10.26,45.11,189.74,84.28,5.10,80.690,7.88,34.39,0.670,11.38
2019-11-21,36.24,15.370,15.760,42.070,7.180,1.500,14.79,32.65,106.20,28.84,17.200,9.02,9.30,4.48,1.81,NaN,17.780,39.26,19.860,68.92,52.78,37.40,57.79,9.29,13.91,9.34,5.340,29.52,12.93,87.90,14.410,20.270,72.76,22.850,16.84,90.130,11.15,32.05,12.28,52.42,10.13,44.48,195.08,84.24,5.06,79.840,7.71,34.25,0.680,11.39
2019-11-22,36.63,15.330,15.930,40.600,7.060,1.520,14.71,32.61,107.12,28.53,16.930,9.45,9.76,4.50,1.82,NaN,17.880,39.21,19.700,69.99,53.74,38.47,58.90,9.05,14.30,9.34,5.160,30.63,13.08,88.89,14.450,19.910,71.81,22.910,16.94,90.840,12.05,32.17,12.29,52.92,9.47,44.15,195.54,82.48,4.92,80.140,7.48,33.79,0.720,11.61
2019-11-25,37.32,15.441,16.125,43.415,7.415,1.591,14.94,33.04,109.63,29.28,16.985,9.78,10.05,4.49,1.84,NaN,17.975,39.57,20.045,72.63,55.03,39.25,59.60,9.40,14.96,9.28,5.135,31.54,13.40,98.71,14.695,20.315,72.98,23.019,16.73,91.779,13.20,33.14,12.38,54.12,9.01,44.74,201.49,83.02,4.95,80.895,7.38,34.40,0.793,11.55


Ignore the fact that the `head()` and `tail()` look offset. That's just formating. Now onto...

### The same thing but with findata

In [130]:

ticker_list_2nd = ['MLVF','APYX','ICPT','RH','BMCH','CETV','GLUU','COLB','APPN','PCYO','OPI','ENS','ACIW','BTU','IRT','AVCO','GDP','AVX','HMSY','MGRC','SAIC','BH','PLOW','OOMA','GEOS','BRT','GLOG','RGR','TIPT','SIGA','LRN','ANF','DCOM','BDGE','PTGX','FELE','NPK','FORR','MSGN','SSD','CFMS','BXG','UNT','TRUP','HMTV','ACRE','FSBW','WIFI','CNO','CHSP']

portfolio_data_object_2nd = MarketDataRequest(
    data_source= 'yahoo',
    start_date = 'Jan 01 2009',
    fields=['open','close','adj close'],
    vendor_tickers = ticker_list_2nd,
    vendor_fields=['open','close','adj close'])

df2 = market.fetch_market(portfolio_data_object_2nd)

2019-11-25 12:17:07,302 - findatapy.market.datavendorweb - INFO - Request Yahoo data
[*********************100%***********************]  50 of 50 completed

1 Failed download:
- CHSP: No data found, symbol may be delisted
[*********************100%***********************]  50 of 50 completed

1 Failed download:
- CHSP: No data found, symbol may be delisted
2019-11-25 12:17:14,515 - findatapy.market.datavendorweb - INFO - Completed request from Yahoo.
2019-11-25 12:17:16,549 - findatapy.market.ioengine - WARNING - Couldn't push MarketDataRequest_648__abstract_curve_key-None__category-None__category_key-backtest_default-cat_yahoo_daily_NYC__cut-NYC__data_source-yahoo__environment-backtest__expiry_date-NaT__fields-open_close_adj close__finish_date-2019-11-24 21:01:32_587982__freq-daily__freq_mult-1__gran_freq-None__resample-None__resample_how-last__start_date-2009-01-01 00:00:00__tickers-None__trade_side-trade__vendor_fields-open_close_adj close__vendor_tickers-MLVF_APYX_ICPT_RH_BMCH_CETV

That's just so much faster and cleaner looking.

In [131]:
pd.set_option('display.max_columns', 150)
df2.shape
df2.isna().sum().sum()
df2.head()
df2.tail()

(2743, 150)

96888

,MLVF.open,APYX.open,ICPT.open,RH.open,BMCH.open,CETV.open,GLUU.open,COLB.open,APPN.open,PCYO.open,OPI.open,ENS.open,ACIW.open,BTU.open,IRT.open,AVCO.open,GDP.open,AVX.open,HMSY.open,MGRC.open,SAIC.open,BH.open,PLOW.open,OOMA.open,GEOS.open,BRT.open,GLOG.open,RGR.open,TIPT.open,SIGA.open,LRN.open,ANF.open,DCOM.open,BDGE.open,PTGX.open,FELE.open,NPK.open,FORR.open,MSGN.open,SSD.open,CFMS.open,BXG.open,UNT.open,TRUP.open,HMTV.open,ACRE.open,FSBW.open,WIFI.open,CNO.open,CHSP.open,MLVF.close,APYX.close,ICPT.close,RH.close,BMCH.close,CETV.close,GLUU.close,COLB.close,APPN.close,PCYO.close,OPI.close,ENS.close,ACIW.close,BTU.close,IRT.close,AVCO.close,GDP.close,AVX.close,HMSY.close,MGRC.close,SAIC.close,BH.close,PLOW.close,OOMA.close,GEOS.close,BRT.close,GLOG.close,RGR.close,TIPT.close,SIGA.close,LRN.close,ANF.close,DCOM.close,BDGE.close,PTGX.close,FELE.close,NPK.close,FORR.close,MSGN.close,SSD.close,CFMS.close,BXG.close,UNT.close,TRUP.close,HMTV.close,ACRE.close,FSBW.close,WIFI.close,CNO.close,CHSP.close,MLVF.adj close,APYX.adj close,ICPT.adj close,RH.adj close,BMCH.adj close,CETV.adj close,GLUU.adj close,COLB.adj close,APPN.adj close,PCYO.adj close,OPI.adj close,ENS.adj close,ACIW.adj close,BTU.adj close,IRT.adj close,AVCO.adj close,GDP.adj close,AVX.adj close,HMSY.adj close,MGRC.adj close,SAIC.adj close,BH.adj close,PLOW.adj close,OOMA.adj close,GEOS.adj close,BRT.adj close,GLOG.adj close,RGR.adj close,TIPT.adj close,SIGA.adj close,LRN.adj close,ANF.adj close,DCOM.adj close,BDGE.adj close,PTGX.adj close,FELE.adj close,NPK.adj close,FORR.adj close,MSGN.adj close,SSD.adj close,CFMS.adj close,BXG.adj close,UNT.adj close,TRUP.adj close,HMTV.adj close,ACRE.adj close,FSBW.adj close,WIFI.adj close,CNO.adj close,CHSP.adj close
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2009-01-02,8.559732,NaN,NaN,NaN,NaN,21.760000,0.50,12.00,NaN,2.57,NaN,11.00,5.323333,NaN,NaN,NaN,NaN,7.94,10.500000,21.420000,NaN,101.040665,NaN,NaN,8.745,3.69,NaN,6.05,NaN,3.33,18.100000,23.290001,13.33,18.500000,NaN,14.095,77.110001,27.879999,NaN,27.790001,NaN,3.22,26.680000,NaN,NaN,NaN,NaN,NaN,5.18,NaN,8.559732,NaN,NaN,NaN,NaN,22.730000,0.49,11.81,NaN,2.57,NaN,11.28,5.356667,NaN,NaN,NaN,NaN,8.03,10.523334,21.639999,NaN,106.385818,NaN,NaN,10.230,5.20,NaN,6.27,NaN,3.21,18.709999,24.360001,13.11,19.520000,NaN,14.405,78.230003,28.139999,NaN,27.650000,NaN,3.12,28.629999,NaN,NaN,NaN,NaN,NaN,4.71,NaN,8.310700,NaN,NaN,NaN,NaN,4.601780,0.49,8.614738,NaN,2.57,NaN,10.567901,5.356667,NaN,NaN,NaN,NaN,6.110076,10.523334,15.196320,NaN,106.385818,NaN,NaN,10.230,3.750430,NaN,4.801984,NaN,3.21,18.709999,17.697693,8.761845,13.201606,NaN,12.642794,38.003326,22.649536,NaN,23.224552,NaN,2.857478,28.629999,NaN,NaN,NaN,NaN,NaN,4.188087,NaN
2009-01-05,8.373651,NaN,NaN,NaN,NaN,22.490000,0.53,11.79,NaN,2.46,NaN,11.29,5.353333,NaN,NaN,NaN,NaN,8.06,10.583333,21.730000,NaN,107.765213,NaN,NaN,10.425,5.04,NaN,6.23,NaN,3.26,18.750000,24.170000,13.14,18.750000,NaN,14.515,78.230003,28.070000,NaN,27.540001,NaN,3.10,28.620001,NaN,NaN,NaN,NaN,NaN,4.41,NaN,8.373651,NaN,NaN,NaN,NaN,21.549999,0.50,11.20,NaN,2.50,NaN,11.30,5.416667,NaN,NaN,NaN,NaN,7.85,10.390000,22.280001,NaN,115.524315,NaN,NaN,10.875,4.75,NaN,6.31,NaN,3.28,18.740000,24.580000,12.88,18.750000,NaN,14.770,77.879997,26.250000,NaN,26.809999,NaN,2.91,29.870001,NaN,NaN,NaN,NaN,NaN,4.45,NaN,8.130033,NaN,NaN,NaN,NaN,4.362884,0.50,8.169774,NaN,2.50,NaN,10.586638,5.416667,NaN,NaN,NaN,NaN,5.973112,10.390000,15.645757,NaN,115.524315,NaN,NaN,10.875,3.425874,NaN,4.832619,NaN,3.28,18.740000,17.857525,8.608127,12.680840,NaN,12.963145,37.833302,21.128296,NaN,22.518999,NaN,2.665147,29.870001,NaN,NaN,NaN,NaN,NaN,3.956898,NaN
2009-01-06,8.373651,NaN,NaN,NaN,NaN,22.299999,0.52,11.31,NaN,2.60,NaN,11.40,5.470000,NaN,NaN,NaN,NaN,8.07,10.603333,22.420000,NaN,115.869164,NaN,NaN,10.950,4.67,NaN,6.31,NaN,3.28,18.520000,24.500000,13.01,19.000000,NaN,14.790,78.27999

,MLVF.open,APYX.open,ICPT.open,RH.open,BMCH.open,CETV.open,GLUU.open,COLB.open,APPN.open,PCYO.open,OPI.open,ENS.open,ACIW.open,BTU.open,IRT.open,AVCO.open,GDP.open,AVX.open,HMSY.open,MGRC.open,SAIC.open,BH.open,PLOW.open,OOMA.open,GEOS.open,BRT.open,GLOG.open,RGR.open,TIPT.open,SIGA.open,LRN.open,ANF.open,DCOM.open,BDGE.open,PTGX.open,FELE.open,NPK.open,FORR.open,MSGN.open,SSD.open,CFMS.open,BXG.open,UNT.open,TRUP.open,HMTV.open,ACRE.open,FSBW.open,WIFI.open,CNO.open,CHSP.open,MLVF.close,APYX.close,ICPT.close,RH.close,BMCH.close,CETV.close,GLUU.close,COLB.close,APPN.close,PCYO.close,OPI.close,ENS.close,ACIW.close,BTU.close,IRT.close,AVCO.close,GDP.close,AVX.close,HMSY.close,MGRC.close,SAIC.close,BH.close,PLOW.close,OOMA.close,GEOS.close,BRT.close,GLOG.close,RGR.close,TIPT.close,SIGA.close,LRN.close,ANF.close,DCOM.close,BDGE.close,PTGX.close,FELE.close,NPK.close,FORR.close,MSGN.close,SSD.close,CFMS.close,BXG.close,UNT.close,TRUP.close,HMTV.close,ACRE.close,FSBW.close,WIFI.close,CNO.close,CHSP.close,MLVF.adj close,APYX.adj close,ICPT.adj close,RH.adj close,BMCH.adj close,CETV.adj close,GLUU.adj close,COLB.adj close,APPN.adj close,PCYO.adj close,OPI.adj close,ENS.adj close,ACIW.adj close,BTU.adj close,IRT.adj close,AVCO.adj close,GDP.adj close,AVX.adj close,HMSY.adj close,MGRC.adj close,SAIC.adj close,BH.adj close,PLOW.adj close,OOMA.adj close,GEOS.adj close,BRT.adj close,GLOG.adj close,RGR.adj close,TIPT.adj close,SIGA.adj close,LRN.adj close,ANF.adj close,DCOM.adj close,BDGE.adj close,PTGX.adj close,FELE.adj close,NPK.adj close,FORR.adj close,MSGN.adj close,SSD.adj close,CFMS.adj close,BXG.adj close,UNT.adj close,TRUP.adj close,HMTV.adj close,ACRE.adj close,FSBW.adj close,WIFI.adj close,CNO.adj close,CHSP.adj close
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-11-18,22.500000,7.53,81.849998,188.429993,28.959999,4.43,5.64,39.599998,43.980000,11.89,32.020000,67.959999,33.590000,9.54,14.66,1.600,9.01,14.96,28.459999,72.930000,82.559998,108.800003,53.090000,11.74,13.48,17.129999,9.64,46.290001,7.57,5.27,20.139999,18.389999,19.750000,33.040001,10.60,53.049999,90.510002,36.910000,18.059999,81.370003,1.90,9.540,0.83,32.090000,12.47,15.37,59.509998,12.10,18.209999,NaN,23.010000,6.99,83.010002,187.550003,29.059999,4.46,5.46,39.799999,43.450001,11.62,32.340000,67.599998,33.700001,8.99,14.66,1.59,9.59,14.73,28.750000,73.010002,83.680000,107.300003,54.009998,11.63,13.74,16.900000,9.26,45.799999,7.73,5.14,20.129999,17.080000,19.990000,32.939999,10.20,53.110001,88.500000,37.049999,18.490000,81.250000,1.76,9.48,0.81,33.389999,12.62,15.39,59.669998,11.75,18.350000,NaN,23.010000,6.99,83.010002,187.550003,29.059999,4.46,5.46,39.799999,43.450001,11.62,32.340000,67.599998,33.700001,8.99,14.66,1.59,9.59,14.73,28.750000,73.010002,83.680000,107.300003,54.009998,11.63,13.74,16.900000,9.26,45.799999,7.73,5.14,20.129999,17.080000,19.990000,32.939999,10.20,53.110001,88.500000,37.049999,18.490000,81.250000,1.76,9.48,0.81,33.389999,12.62,15.39,59.669998,11.75,18.350000,NaN
2019-11-19,23.129999,7.05,83.529999,187.259995,29.040001,4.46,5.47,39.939999,43.509998,11.65,32.380001,67.980003,33.889999,9.06,14.67,1.600,9.40,14.89,28.900000,73.389999,84.010002,107.099998,54.180000,11.70,13.70,16.969999,9.25,45.680000,7.72,5.15,20.180000,16.700001,20.100000,33.029999,10.38,53.340000,88.750000,37.290001,17.049999,81.690002,1.75,9.510,0.80,33.310001,12.68,15.44,59.599998,11.74,18.420000,NaN,23.379999,6.75,85.550003,182.220001,29.049999,4.47,5.35,39.889999,43.560001,11.86,32.169998,68.650002,36.389999,9.06,14.75,1.57,9.21,14.84,28.750000,75.110001,84.029999,105.230003,54.000000,11.42,14.92,17.170000,8.92,45.389999,7.89,5.08,19.660000,16.799999,20.190001,33.000000,10.64,53.310001,89.419998,37.610001,17.900000,80.980003,1.80,9.58,0.70,34.430000,12.94,15.55,59.380001,11.35,18.059999,NaN,23.379999,6.75,85.550003,182.220001,29.049999,4.47,5.35,39.889999,43.560001

All in all, findata is pretty neat.


# Showcasing everything you SHOULD NOT do

The following examples are meant to showcase when problems occur. As for why they occur, I will be putting up another notebook in the future explaining what is going on behind the scenes.

In [152]:
# this should throw a ton of errors... don't actually run this btw
#portfolio_data_object_error = MarketDataRequest(
#    data_source= 'yahoo',
#    vendor_tickers = ticker_list_2nd)
#    start_date = 'year',
# notice I completely ommited the vendorfield attribute and the field attribute.
#df_error = market.fetch_market(portfolio_data_object_error)

ion, did you type it correctly: low, using 'close' as default.
2019-11-26 12:44:12,107 - findatapy.market.datavendor - ERROR - Couldn't find field conversion, did you type it correctly: low, using 'close' as default.
2019-11-26 12:44:12,107 - findatapy.market.datavendor - ERROR - Couldn't find field conversion, did you type it correctly: low, using 'close' as default.
2019-11-26 12:44:12,108 - findatapy.market.datavendor - ERROR - Couldn't find field conversion, did you type it correctly: low, using 'close' as default.
2019-11-26 12:44:12,108 - findatapy.market.datavendor - ERROR - Couldn't find field conversion, did you type it correctly: low, using 'close' as default.
2019-11-26 12:44:12,109 - findatapy.market.datavendor - ERROR - Couldn't find field conversion, did you type it correctly: low, using 'close' as default.
2019-11-26 12:44:12,109 - findatapy.market.datavendor - ERROR - Couldn't find field conversion, did you type it correctly: low, using 'close' as default.
2019-11-26 12

Notice all the errors

In [153]:
df_error.shape
df_error.isna().sum().sum()
df_error.head()
df_error.tail()

(250, 300)

1638

,MLVF.close,MLVF.close,MLVF.close,MLVF.close,MLVF.close,MLVF.close,APYX.close,APYX.close,APYX.close,APYX.close,APYX.close,APYX.close,ICPT.close,ICPT.close,ICPT.close,ICPT.close,ICPT.close,ICPT.close,RH.close,RH.close,RH.close,RH.close,RH.close,RH.close,BMCH.close,BMCH.close,BMCH.close,BMCH.close,BMCH.close,BMCH.close,CETV.close,CETV.close,CETV.close,CETV.close,CETV.close,CETV.close,GLUU.close,GLUU.close,GLUU.close,GLUU.close,GLUU.close,GLUU.close,COLB.close,COLB.close,COLB.close,COLB.close,COLB.close,COLB.close,APPN.close,APPN.close,APPN.close,APPN.close,APPN.close,APPN.close,PCYO.close,PCYO.close,PCYO.close,PCYO.close,PCYO.close,PCYO.close,OPI.close,OPI.close,OPI.close,OPI.close,OPI.close,OPI.close,ENS.close,ENS.close,ENS.close,ENS.close,ENS.close,ENS.close,ACIW.close,ACIW.close,ACIW.close,...,FORR.close,FORR.close,FORR.close,MSGN.close,MSGN.close,MSGN.close,MSGN.close,MSGN.close,MSGN.close,SSD.close,SSD.close,SSD.close,SSD.close,SSD.close,SSD.close,CFMS.close,CFMS.close,CFMS.close,CFMS.close,CFMS.close,CFMS.close,BXG.close,BXG.close,BXG.close,BXG.close,BXG.close,BXG.close,UNT.close,UNT.close,UNT.close,UNT.close,UNT.close,UNT.close,TRUP.close,TRUP.close,TRUP.close,TRUP.close,TRUP.close,TRUP.close,HMTV.close,HMTV.close,HMTV.close,HMTV.close,HMTV.close,HMTV.close,ACRE.close,ACRE.close,ACRE.close,ACRE.close,ACRE.close,ACRE.close,FSBW.close,FSBW.close,FSBW.close,FSBW.close,FSBW.close,FSBW.close,WIFI.close,WIFI.close,WIFI.close,WIFI.close,WIFI.close,WIFI.close,CNO.close,CNO.close,CNO.close,CNO.close,CNO.close,CNO.close,CHSP.close,CHSP.close,CHSP.close,CHSP.close,CHSP.close,CHSP.close
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-11-27,20.840000,20.840000,21.139999,20.500000,20.709999,20500.0,NaN,NaN,NaN,NaN,NaN,NaN,108.660004,108.660004,112.870003,108.019997,110.110001,292300.0,110.220001,110.220001,115.830002,109.629997,114.129997,605900.0,16.530001,16.530001,16.590000,16.139999,16.280001,493600.0,3.25,3.25,3.32,3.22,3.30,96100.0,6.87,6.87,6.93,6.58,6.80,1952200.0,37.851685,39.299999,39.730000,39.000000,39.369999,170800.0,27.620001,27.620001,27.740000,26.228001,26.980000,437700.0,10.33,10.33,10.39,10.160,10.16,7300.0,30.868753,33.279999,33.680000,33.000000,33.560001,311400.0,82.692017,83.540001,83.949997,83.019997,83.269997,154800.0,28.080000,28.080000,28.980000,...,44.270000,45.639999,17400.0,26.910000,26.910000,27.469999,26.680000,26.889999,402700.0,55.618294,56.439999,56.869999,56.230000,56.470001,159900.0,0.481,0.481,0.516,0.460,0.516,271300.0,13.325203,14.08,14.45,12.050,12.05,188800.0,20.520000,20.520000,21.230000,20.360001,20.590000,283200.0,26.910000,26.910000,27.670000,26.330,27.090000,274500.0,13.82,13.82,14.01,13.59,13.86,38900.0,12.985682,14.19,14.27,14.15,14.27,72500.0,46.576134,47.169998,47.400002,46.000000,46.200001,21500.0,25.100000,25.100000,25.520000,24.510000,25.209999,443600.0,17.503767,17.969999,18.360001,17.950001,18.139999,684600.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-28,20.879999,20.879999,20.980000,20.389999,20.840000,29900.0,NaN,NaN,NaN,NaN,NaN,NaN,114.059998,114.059998,114.339996,106.430000,108.739998,409600.0,116.610001,116.610001,118.470001,110.510002,110.720001,976900.0,17.020000,17.020000,17.080000,16.290001,16.600000,293200.0,3.34,3.34,3.35,3.27,3.28,158600.0,7.21,7.21,7.25,6.86,6.93,1716600.0,38.834099,40.320000,40.520000,39.000000,39.349998,430300.0,29.090000,29.090000,29.629999,27.950001,28.000000,502200.0,10.55,10.55,10.83,10.070,10.40,28700.0,31.796303,34.279999,34.520000,32.759998,33.200001,398300.0,85.037956,85.910004,85.930000,82.449997,84.169998,258000.0,29.320000,29.320000,29.520000,...,44.740002,45.389999,40300.0,27.490000,27.490000,27.559999,26.420000,27.040001,259400.0,56.938789,57.779999,57.950001,55.630001,56.470001,289500.0,0.504,0.504,0.510,0.451,0.481,178300.0,13.561801,14.33,14.97,13.810,14.27,132300.0,21.150000,21.150000,21.200001,20.190001,20.540001,262400.0,28.629999,2

,MLVF.close,MLVF.close,MLVF.close,MLVF.close,MLVF.close,MLVF.close,APYX.close,APYX.close,APYX.close,APYX.close,APYX.close,APYX.close,ICPT.close,ICPT.close,ICPT.close,ICPT.close,ICPT.close,ICPT.close,RH.close,RH.close,RH.close,RH.close,RH.close,RH.close,BMCH.close,BMCH.close,BMCH.close,BMCH.close,BMCH.close,BMCH.close,CETV.close,CETV.close,CETV.close,CETV.close,CETV.close,CETV.close,GLUU.close,GLUU.close,GLUU.close,GLUU.close,GLUU.close,GLUU.close,COLB.close,COLB.close,COLB.close,COLB.close,COLB.close,COLB.close,APPN.close,APPN.close,APPN.close,APPN.close,APPN.close,APPN.close,PCYO.close,PCYO.close,PCYO.close,PCYO.close,PCYO.close,PCYO.close,OPI.close,OPI.close,OPI.close,OPI.close,OPI.close,OPI.close,ENS.close,ENS.close,ENS.close,ENS.close,ENS.close,ENS.close,ACIW.close,ACIW.close,ACIW.close,...,FORR.close,FORR.close,FORR.close,MSGN.close,MSGN.close,MSGN.close,MSGN.close,MSGN.close,MSGN.close,SSD.close,SSD.close,SSD.close,SSD.close,SSD.close,SSD.close,CFMS.close,CFMS.close,CFMS.close,CFMS.close,CFMS.close,CFMS.close,BXG.close,BXG.close,BXG.close,BXG.close,BXG.close,BXG.close,UNT.close,UNT.close,UNT.close,UNT.close,UNT.close,UNT.close,TRUP.close,TRUP.close,TRUP.close,TRUP.close,TRUP.close,TRUP.close,HMTV.close,HMTV.close,HMTV.close,HMTV.close,HMTV.close,HMTV.close,ACRE.close,ACRE.close,ACRE.close,ACRE.close,ACRE.close,ACRE.close,FSBW.close,FSBW.close,FSBW.close,FSBW.close,FSBW.close,FSBW.close,WIFI.close,WIFI.close,WIFI.close,WIFI.close,WIFI.close,WIFI.close,CNO.close,CNO.close,CNO.close,CNO.close,CNO.close,CNO.close,CHSP.close,CHSP.close,CHSP.close,CHSP.close,CHSP.close,CHSP.close
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-11-18,23.010000,23.010000,23.170000,22.500000,22.500000,8800.0,6.99,6.99,7.530,6.97,7.53,103200.0,83.010002,83.010002,83.029999,80.250000,81.849998,368800.0,187.550003,187.550003,192.740005,186.869995,188.429993,1314900.0,29.059999,29.059999,29.170000,28.830000,28.959999,232700.0,4.46,4.46,4.46,4.43,4.43,541200.0,5.46,5.46,5.65,5.40,5.64,3406200.0,39.799999,39.799999,39.820000,39.250000,39.599998,191900.0,43.450001,43.450001,44.250000,43.243000,43.980000,460700.0,11.62,11.62,11.975,11.530,11.89,42100.0,32.340000,32.340000,32.360001,31.940001,32.020000,130400.0,67.599998,67.599998,68.000000,67.220001,67.959999,134600.0,33.700001,33.700001,33.880001,...,36.500000,36.910000,41900.0,18.490000,18.490000,18.500000,17.990000,18.059999,727700.0,81.250000,81.250000,82.110001,80.730003,81.370003,372400.0,1.76,1.76,1.90,1.760,1.90,543800.0,9.48,9.48,9.54,9.320,9.540,38300.0,0.81,0.81,0.88,0.77,0.83,1642300.0,33.389999,33.389999,33.439999,32.090000,32.090000,255600.0,12.62,12.62,12.65,12.43,12.47,14000.0,15.39,15.39,15.54,15.34,15.37,147100.0,59.669998,59.669998,59.889999,59.060001,59.509998,6800.0,11.75,11.75,12.39,11.71,12.10,455700.0,18.350000,18.350000,18.360001,18.040001,18.209999,1633300.0,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-19,23.379999,23.379999,23.430000,23.080000,23.129999,14300.0,6.75,6.75,7.584,6.71,7.05,140100.0,85.550003,85.550003,85.839996,83.139999,83.529999,435500.0,182.220001,182.220001,187.550003,178.500000,187.259995,1201400.0,29.049999,29.049999,29.330000,28.950001,29.040001,198000.0,4.47,4.47,4.48,4.45,4.46,633700.0,5.35,5.35,5.49,5.32,5.47,2836800.0,39.889999,39.889999,40.139999,38.660000,39.939999,214300.0,43.560001,43.560001,43.827000,43.090000,43.509998,425100.0,11.86,11.86,12.200,11.650,11.65,40300.0,32.169998,32.169998,32.560001,32.139999,32.380001,123800.0,68.650002,68.650002,68.709999,67.349998,67.980003,125600.0,36.389999,36.389999,36.529999,...,37.139999,37.290001,68700.0,17.900000,17.900000,18.020000,16.940001,17.049999,1665000.0,80.980003,80.980003,81.730003,80.910004,81.690002,267000.0,1.80,1.80,1.82,1.730,1.75,323400.0,9.58,9.58,9.81,9.510,9.510,24100.0,0.70,0.70,0.81,0.68,0.80,1512400.0,34.430000,34.430000,34.799999,33.310001,33.310001,499700.0,12.94,12.94,13.03,12.68,12.6

# Fascinating

It returned data but I can't even make sense of the data. It seems that it attempted to download 6 fields per ticker. I suspect these fields are the `open`, `close`, `high`, `low`, `adj close`, and `volume` fields in some order.  All those fields were named `close` because that's what findata defaults too when it can't interpret the field attributes correctly.

Actually, we can figure out what order they are in by running the following and comparing the results


In [149]:
# this is suppose to work btw.
data_request_check = MarketDataRequest(
    data_source='yahoo',
    start_date='year',
    fields=['open', 'close', 'high', 'low', 'adj close', 'volume'],
    vendor_tickers=['MLVF'],
    vendor_fields=['Open','Close','High','Low','AdjClose', 'Volume'])
df_request_check = market.fetch_market(data_request_check)

2019-11-26 12:43:28,032 - findatapy.market.datavendorweb - INFO - Request Yahoo data
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
2019-11-26 12:43:28,356 - findatapy.market.datavendorweb - INFO - Completed request from Yahoo.
2019-11-26 12:43:30,365 - findatapy.market.ioengine - WARNING - Couldn't push MarketDataRequest_460__abstract_curve_key-None__category-None__category_key-backtest_default-cat_yahoo_daily_NYC__cut-NYC__data_source-yahoo__environment-backtest__expiry_date-NaT__fields-open_close_high_low_adj close_volume__finish_date-2019-11-24 21:01:32_587982__freq-daily__freq_mult-1__gran_freq-None__resample-None__resample_how-last__start_date-2018-11-26 17:43:28_030455__tickers-None__trade_side-trade__vendor_fields-Open_Close_High_Low_AdjClose_Volume__vendor_tickers-MLVF to Redis: Error 10061 connecting to 127.0.0.1:6379. No connection could be made because the target machine actively refu

In [151]:
df_request_check.head()

,MLVF.open,MLVF.close,MLVF.high,MLVF.low,MLVF.adj close,MLVF.volume
Date,,,,,,
2018-11-27,20.709999,20.840000,21.139999,20.500000,20.840000,20500.0
2018-11-28,20.840000,20.879999,20.980000,20.389999,20.879999,29900.0
2018-11-29,20.740000,20.799999,20.900000,20.639999,20.799999,8100.0
2018-11-30,20.820000,20.900000,21.000000,20.820000,20.900000,13400.0
2018-12-03,21.070000,21.219999,21.230000,20.799999,21.219999,6400.0


# Now back to creating errors

What happens when you exclude `fields` and `vendor_fields` attributes?

In [189]:
# excluding 'fields' and 'vendor_fields'
data_request_error = MarketDataRequest(
    data_source='yahoo',
    start_date='year',
    vendor_tickers=['FB'])
df_error_2 = market.fetch_market(data_request_error)
df_error_2.tail()

2019-11-26 14:16:43,026 - findatapy.market.datavendorweb - INFO - Request Yahoo data
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
2019-11-26 14:16:43,437 - findatapy.market.datavendor - ERROR - Couldn't find field conversion, did you type it correctly: open, using 'close' as default.
2019-11-26 14:16:43,438 - findatapy.market.datavendor - ERROR - Couldn't find field conversion, did you type it correctly: high, using 'close' as default.
2019-11-26 14:16:43,439 - findatapy.market.datavendor - ERROR - Couldn't find field conversion, did you type it correctly: low, using 'close' as default.
2019-11-26 14:16:43,440 - findatapy.market.datavendor - ERROR - Couldn't find field conversion, did you type it correctly: close, using 'close' as default.
2019-11-26 14:16:43,441 - findatapy.market.datavendor - ERROR - Couldn't find field conversion, did you type it correctly: adj close, using 'close' as defaul

,FB.close,FB.close,FB.close,FB.close,FB.close,FB.close
Date,,,,,,
2019-11-18,194.559998,198.630005,193.050003,197.399994,197.399994,16167200.0
2019-11-19,197.399994,200.000000,196.860001,199.320007,199.320007,19056800.0
2019-11-20,198.580002,199.589996,195.429993,197.509995,197.509995,12355400.0
2019-11-21,197.419998,199.089996,196.860001,197.929993,197.929993,12131000.0
2019-11-22,198.380005,199.300003,197.619995,198.820007,198.820007,9959800.0


Looks like we got the 6 fields that yahoo provides, but without the field names. In order of appearance I know they are, `open`, `high`, `low`, `close`, `adj close`, and `volume`. Why do they appear in this order. Because the source code says so.

What happens when you exclude the `data_source` attribute (hint: it won't work)

In [168]:
data_request_source_error = MarketDataRequest(
    data_source=None,
    start_date='year',
    fields=['open', 'close', 'high', 'low', 'adj close', 'volume'],
    vendor_tickers=['MLVF'],
    vendor_fields=['Open','Close','High','Low','AdjClose', 'Volume'])
df_source_error = market.fetch_market(data_request_source_error)

2019-11-26 13:31:02,018 - findatapy.market.marketdatagenerator - ERROR - Was data source specified?


AttributeError: 'NoneType' object has no attribute 'load_ticker'

### Hmm... not the error I was expecting... guess I need to take a closer look at source code...
***
What happens when you exclude `vendor_tickers` and rely instead on the `ticker` attribute?

In [180]:
# this cell is suppose to fail
data_request_no_vendor_tickers = MarketDataRequest(
    data_source='yahoo',
    start_date='year',
    tickers=['MLVF'],
    fields=['open', 'close', 'high', 'low', 'adj close', 'volume'],
    vendor_fields=['Open','Close','High','Low','AdjClose', 'Volume'])
df_no_vendor_tickers = market.fetch_market(data_request_no_vendor_tickers)
df_no_vendor_tickers

2019-11-26 13:41:41,693 - findatapy.market.datavendor - ERROR - Couldn't find ticker conversion, did you type it correctly: MLVF
2019-11-26 13:41:41,695 - findatapy.market.datavendorweb - INFO - Request Yahoo data


TypeError: can only join an iterable

It won't work becaue the name `tickers` is a misnomer. Each ticker in `tickers` is mapped to the `vendor_tickers` list. This can be useful for giving each ticker it's actual company name.... but can also be used for hilarious effect. Observe.

In [185]:
data_request_mismatch = MarketDataRequest(
    data_source='yahoo',
    start_date='year',
    tickers=['Alphabet.A', 'facebook'], 
    fields=['open', 'close'],
    vendor_tickers=['FB','GOOGL'], # switched the tickers. The data is correct but the indexs are mislabeled.
    vendor_fields=['Open','Close'])
df_mismatch = market.fetch_market(data_request_mismatch)
df_mismatch.tail()

2019-11-26 13:56:41,656 - findatapy.market.datavendorweb - INFO - Request Yahoo data
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
2019-11-26 13:56:42,128 - findatapy.market.datavendorweb - INFO - Completed request from Yahoo.
2019-11-26 13:56:44,149 - findatapy.market.ioengine - WARNING - Couldn't push MarketDataRequest_428__abstract_curve_key-None__category-None__category_key-backtest_default-cat_yahoo_daily_NYC__cut-NYC__data_source-yahoo__environment-backtest__expiry_date-NaT__fields-open_close__finish_date-2019-11-24 21:01:32_587982__freq-daily__freq_mult-1__gran_freq-None__resample-None__resample_how-last__start_date-2018-11-26 18:56:41_653081__tickers-Alphabet_A_facebook__trade_side-trade__vendor_fields-Open_Close__vendor_tickers-FB_GOOGL to Redis: Error 10061 connecting to 127.0.0.1:6379. No connection could be made because the target machine actively refused it.


,Alphabet.A.open,facebook.open,Alphabet.A.close,facebook.close
Date,,,,
2019-11-18,194.559998,1332.339966,197.399994,1319.839966
2019-11-19,197.399994,1326.630005,199.320007,1312.589966
2019-11-20,198.580002,1311.359985,197.509995,1301.859985
2019-11-21,197.419998,1299.250000,197.929993,1300.140015
2019-11-22,198.380005,1303.000000,198.820007,1293.670044


This in a nutshell shows why we have to be careful with our naming conventions. And also how our assumptions can be very wrong. When I first used findata I assumed that the `tickers` attributes was the place to drop the literal tickers. But it turns out those go in `vendor_tickers`.

Now onto figuring out what is going on with the `fields` and `vendor fields` attributes. 

In [187]:
# this won't return what we want.
data_request_no_fields = MarketDataRequest(
    data_source='yahoo',
    start_date='year',
    vendor_tickers= ['FB','GOOGL'],
    vendor_fields=['Open','Close','High','Low','AdjClose', 'Volume'])
df_no_fields = market.fetch_market(data_request_no_fields)
df_no_fields.tail()

2019-11-26 14:11:33,241 - findatapy.market.datavendorweb - INFO - Request Yahoo data
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
2019-11-26 14:11:33,633 - findatapy.market.datavendorweb - INFO - Completed request from Yahoo.
2019-11-26 14:11:35,640 - findatapy.market.ioengine - WARNING - Couldn't push MarketDataRequest_433__abstract_curve_key-None__category-None__category_key-backtest_default-cat_yahoo_daily_NYC__cut-NYC__data_source-yahoo__environment-backtest__expiry_date-NaT__fields-close__finish_date-2019-11-24 21:01:32_587982__freq-daily__freq_mult-1__gran_freq-None__resample-None__resample_how-last__start_date-2018-11-26 19:11:33_239758__tickers-None__trade_side-trade__vendor_fields-Open_Close_High_Low_AdjClose_Volume__vendor_tickers-FB_GOOGL to Redis: Error 10061 connecting to 127.0.0.1:6379. No connection could be made because the target machine actively refused it.


,FB.close,FB.close,GOOGL.close,GOOGL.close
Date,,,,
2019-11-18,197.399994,194.559998,1319.839966,1332.339966
2019-11-19,199.320007,197.399994,1312.589966,1326.630005
2019-11-20,197.509995,198.580002,1301.859985,1311.359985
2019-11-21,197.929993,197.419998,1300.140015,1299.250000
2019-11-22,198.820007,198.380005,1293.670044,1303.000000


Hmmm... no errors were thrown. But we didn't get back what we wanted. We did get 2 results per day per ticker. On closer inspection the first field is indeed `close` and the 2nd field is `open`. Which is... not what I expected at all. In short I should revist the source code.

I'm done testing this stuff. I like findata.py. I'm going to use it for some of my underlying code.